## Installation

In [36]:
pip install wandb numpy pandas matplotlib

## Q1: fashion-MNIST dataset

In [37]:
import wandb
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

classes = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

def logClassImages(project_name:str):
  wandb.init(project=project_name)
  wandb_image_indices = []

  for classNumber in range(10):
    for j in range(len(y_test)):
      if y_test[j] == classNumber:
        wandb_image_indices.append(x_test[j])
        break

  wandb_images = [wandb.Image(wandb_image_indices[i], caption = classes[i]) for i in range(10)]
  wandb.log({"Sample images for each class": wandb_images})
  wandb.finish()

# logClassImages("da6401_assignment1")

## Feedforward neural network



### Libraries required

In [38]:
import wandb
from keras.datasets import fashion_mnist
import numpy as np
import copy

### Activation functions

In [39]:
"""
  ACTIVATION FUNCTIONS
"""
def identity(x):
    return x

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Numerical Stability
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

### Loss Functions

In [72]:
"""
  LOSS FUNCTIONS
"""
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

import numpy as np

def cross_entropy_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-9), axis=-1)    # 1e-9 to avy_pred + 1e-9oid log 0

### Derivatives

In [73]:
"""
  DERIVATIVES OF ACTIVATION AND LOSS FUNCTIONS
"""
def identity_derivative(x):
    return np.ones_like(x)

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def mean_squared_error_derivative(y_true, y_pred):
    return y_pred - y_true

def cross_entropy_loss_derivative(y_true, y_pred):
    return -y_true / (y_pred + 1e-9)

def softmax_derivative(inp:np.array):
    derivates = []
    if(len(inp.shape) == 1):
      S_vector = inp.reshape(-1, 1)
      derivates = np.diag(inp) - np.dot(S_vector, S_vector.T)
    elif(len(inp.shape) == 2):
      for i in range(inp.shape[0]):
        S_vector = inp[i].reshape(-1, 1)
        derivates.append(np.diag(inp[i]) - np.dot(S_vector, S_vector.T))

    return np.array(derivates)

### Optimizers

In [74]:
"""
  OPTIMIZERS UPDATE RULES
"""

# STOCHASTIC GRADIENT DESCENT
def sgd(optimizer_input_dict, wts_bias_history_dict, itr=None):
  # cant update weights in one single matrix op as dimensions of weights can be different in each layer
  for i in range(optimizer_input_dict["n_hiddenLayers"]+1):
    # weight decay term added additionally to the formula in slides
    wts_bias_history_dict["weights"][i] = wts_bias_history_dict["weights"][i] - optimizer_input_dict["learning_rate"] * (wts_bias_history_dict["dw"][i] + (optimizer_input_dict["weight_decay"] * wts_bias_history_dict["weights"][i]))
    wts_bias_history_dict["biases"][i] = wts_bias_history_dict["biases"][i] - optimizer_input_dict["learning_rate"] * wts_bias_history_dict["db"][i]

# MOMENTUM BASED GRADIENT DESCENT
def momentumGradientDescent(optimizer_input_dict, wts_bias_history_dict, itr=None):
  for i in range(optimizer_input_dict["n_hiddenLayers"]+1):
    wts_bias_history_dict["history_weights"][i] = (optimizer_input_dict["momentum"] * wts_bias_history_dict["history_weights"][i]) + wts_bias_history_dict["dw"][i] + (optimizer_input_dict["weight_decay"] * wts_bias_history_dict["weights"][i])
    wts_bias_history_dict["weights"][i] = wts_bias_history_dict["weights"][i] - optimizer_input_dict["learning_rate"] * wts_bias_history_dict["history_weights"][i]

    wts_bias_history_dict["history_biases"][i] = (optimizer_input_dict["momentum"] * wts_bias_history_dict["history_biases"][i]) + wts_bias_history_dict["db"][i]
    wts_bias_history_dict["biases"][i] = wts_bias_history_dict["biases"][i] - optimizer_input_dict["learning_rate"] * wts_bias_history_dict["history_biases"][i]

# NAG(NESTEROV ACCELERATED GRADIENT DESCENT)
def nag(optimizer_input_dict, wts_bias_history_dict, itr=None):
  for i in range(optimizer_input_dict["n_hiddenLayers"]+1):
    # dw,db will contain lookahead gradients only since forward and backward propagations are implemented accordingly
    wts_bias_history_dict["history_weights"][i] = (optimizer_input_dict["momentum"] * wts_bias_history_dict["history_weights"][i]) + wts_bias_history_dict["dw"][i] + (optimizer_input_dict["weight_decay"] * wts_bias_history_dict["weights"][i])
    wts_bias_history_dict["weights"][i] = wts_bias_history_dict["weights"][i] - optimizer_input_dict["learning_rate"] * wts_bias_history_dict["history_weights"][i]

    wts_bias_history_dict["history_biases"][i] = (optimizer_input_dict["momentum"] * wts_bias_history_dict["history_biases"][i]) + wts_bias_history_dict["db"][i]
    wts_bias_history_dict["biases"][i] = wts_bias_history_dict["biases"][i] - optimizer_input_dict["learning_rate"] * wts_bias_history_dict["history_biases"][i]

# RMSPROP
def rmsProp(optimizer_input_dict, wts_bias_history_dict, itr=None):
  for i in range(optimizer_input_dict["n_hiddenLayers"]+1):
    wts_bias_history_dict["history_weights"][i] = (optimizer_input_dict["beta"] * wts_bias_history_dict["history_weights"][i]) + (1 - optimizer_input_dict["beta"]) * (wts_bias_history_dict["dw"][i] ** 2)
    wts_bias_history_dict["weights"][i] = wts_bias_history_dict["weights"][i] - ((optimizer_input_dict["learning_rate"]/np.sqrt(wts_bias_history_dict["history_weights"][i] + optimizer_input_dict["epsilon"])) * (wts_bias_history_dict["dw"][i] + (optimizer_input_dict["weight_decay"] * wts_bias_history_dict["weights"][i])))

    wts_bias_history_dict["history_biases"][i] = (optimizer_input_dict["beta"] * wts_bias_history_dict["history_biases"][i]) + (1 - optimizer_input_dict["beta"]) * (wts_bias_history_dict["db"][i] ** 2)
    wts_bias_history_dict["biases"][i] = wts_bias_history_dict["biases"][i] - ((optimizer_input_dict["learning_rate"]/np.sqrt(wts_bias_history_dict["history_biases"][i] + optimizer_input_dict["epsilon"])) * (wts_bias_history_dict["db"][i]))

# ADAM
def adam(optimizer_input_dict, wts_bias_history_dict, itr=None):
  for i in range(optimizer_input_dict["n_hiddenLayers"]+1):
    wts_bias_history_dict["history_weights"][i] = (optimizer_input_dict["beta1"] * wts_bias_history_dict["history_weights"][i]) + (1 - optimizer_input_dict["beta1"]) * (wts_bias_history_dict["dw"][i])
    wts_bias_history_dict["second_history_weights"][i] = (optimizer_input_dict["beta2"] * wts_bias_history_dict["second_history_weights"][i]) + (1 - optimizer_input_dict["beta2"]) * (wts_bias_history_dict["dw"][i] ** 2)

    history_weights_hat = wts_bias_history_dict["history_weights"][i] / (1 - (optimizer_input_dict["beta1"] ** (itr)))
    second_history_weights_hat = wts_bias_history_dict["second_history_weights"][i] / (1 - (optimizer_input_dict["beta2"] ** (itr)))

    wts_bias_history_dict["weights"][i] = wts_bias_history_dict["weights"][i] - ((optimizer_input_dict["learning_rate"]/(np.sqrt(second_history_weights_hat) + optimizer_input_dict["epsilon"])) * (history_weights_hat + (optimizer_input_dict["weight_decay"] * wts_bias_history_dict["weights"][i])))

    wts_bias_history_dict["history_biases"][i] = (optimizer_input_dict["beta1"] * wts_bias_history_dict["history_biases"][i]) + (1 - optimizer_input_dict["beta1"]) * wts_bias_history_dict["db"][i]
    wts_bias_history_dict["second_history_biases"][i] = (optimizer_input_dict["beta2"] * wts_bias_history_dict["second_history_biases"][i]) + (1 - optimizer_input_dict["beta2"]) * (wts_bias_history_dict["db"][i] ** 2)

    history_biases_hat = wts_bias_history_dict["history_biases"][i] / (1 - (optimizer_input_dict["beta1"] ** (itr)))
    second_history_biases_hat = wts_bias_history_dict["second_history_biases"][i] / (1 - (optimizer_input_dict["beta2"] ** (itr)))

    wts_bias_history_dict["biases"][i] = wts_bias_history_dict["biases"][i] - ((optimizer_input_dict["learning_rate"]/(np.sqrt(second_history_biases_hat) + optimizer_input_dict["epsilon"])) * (history_biases_hat))

# NADAM
def nadam(optimizer_input_dict, wts_bias_history_dict, itr=None):
  for i in range(optimizer_input_dict["n_hiddenLayers"]+1):
    wts_bias_history_dict["history_weights"][i] = (optimizer_input_dict["beta1"] * wts_bias_history_dict["history_weights"][i]) + (1 - optimizer_input_dict["beta1"]) * (wts_bias_history_dict["dw"][i])
    wts_bias_history_dict["second_history_weights"][i] = (optimizer_input_dict["beta2"] * wts_bias_history_dict["second_history_weights"][i]) + (1 - optimizer_input_dict["beta2"]) * (wts_bias_history_dict["dw"][i] ** 2)

    history_weights_hat = wts_bias_history_dict["history_weights"][i] / (1 - (optimizer_input_dict["beta1"] ** itr))
    second_history_weights_hat = wts_bias_history_dict["second_history_weights"][i] / (1 - (optimizer_input_dict["beta2"] ** itr))

    lookahead_dw = optimizer_input_dict["beta1"] * history_weights_hat + (((1-optimizer_input_dict["beta1"])/(1-(optimizer_input_dict["beta1"] ** itr))) * wts_bias_history_dict["dw"][i])
    wts_bias_history_dict["weights"][i] = wts_bias_history_dict["weights"][i] - ((optimizer_input_dict["learning_rate"]/(np.sqrt(second_history_weights_hat) + optimizer_input_dict["epsilon"])) * (lookahead_dw + (optimizer_input_dict["weight_decay"] * wts_bias_history_dict["weights"][i])))


    wts_bias_history_dict["history_biases"][i] = (optimizer_input_dict["beta1"] * wts_bias_history_dict["history_biases"][i]) + (1 - optimizer_input_dict["beta1"]) * wts_bias_history_dict["db"][i]
    wts_bias_history_dict["second_history_biases"][i] = (optimizer_input_dict["beta2"] * wts_bias_history_dict["second_history_biases"][i]) + (1 - optimizer_input_dict["beta2"]) * (wts_bias_history_dict["db"][i] ** 2)

    history_biases_hat = wts_bias_history_dict["history_biases"][i] / (1 - (optimizer_input_dict["beta1"] ** itr))
    second_history_biases_hat = wts_bias_history_dict["second_history_biases"][i] / (1 - (optimizer_input_dict["beta2"] ** itr))

    lookahead_db = optimizer_input_dict["beta1"] * history_biases_hat + (((1-optimizer_input_dict["beta1"])/(1-(optimizer_input_dict["beta1"] ** itr))) * wts_bias_history_dict["db"][i])
    wts_bias_history_dict["biases"][i] = wts_bias_history_dict["biases"][i] - ((optimizer_input_dict["learning_rate"]/(np.sqrt(second_history_biases_hat) + optimizer_input_dict["epsilon"])) * (lookahead_db))


### Network (forward and back propagation)

In [75]:
class FeedForwardNeuralNetwork:
  # class variables
  optimizersMap = {"sgd": sgd, "momentum": momentumGradientDescent, "nag": nag, "rmsprop": rmsProp, "adam": adam, "nadam": nadam}
  lossFunctionsMap = {"mean_squared_error": mean_squared_error, "cross_entropy" : cross_entropy_loss}
  activationFunctionsMap = {"identity":identity, "sigmoid":sigmoid, "tanh":tanh, "ReLU":relu, "softmax": softmax}
  derivatesFuncMap = {"mean_squared_error": mean_squared_error_derivative, "cross_entropy_loss": cross_entropy_loss_derivative, "identity": identity_derivative,
                      "sigmoid": sigmoid_derivative, "tanh": tanh_derivative, "relu": relu_derivative, "softmax": softmax_derivative}

  def __init__(self,
               input_size=784, output_size=10,
               n_hiddenLayers=3, n_neuronsPerLayer=32,
               activationFun="sigmoid",
               weight_init="random",
               batch_size=64,
               lossFunc="cross_entropy",
               optimizer="adam",
               learning_rate=0.001,
               momentum=0.5,
               beta=0.9, beta1=0.9, beta2=0.99,
               epsilon=1e-8, weight_decay=0.01,
               epochs=10):

    # Inialtization parameters
    self.input_size = input_size  # no of features
    self.output_size = output_size
    self.n_hiddenLayers = n_hiddenLayers
    self.n_neuronsPerLayer = n_neuronsPerLayer
    self.weight_init = weight_init
    self.epochs = epochs

    self.activationFun = FeedForwardNeuralNetwork.activationFunctionsMap[activationFun]
    self.lossFunc = FeedForwardNeuralNetwork.lossFunctionsMap[lossFunc]
    self.optimizer = FeedForwardNeuralNetwork.optimizersMap[optimizer]

    # paramters required for optimizers
    self.batch_size = batch_size
    self.isLookAhead = False;

    if(optimizer == "nag"):
      self.isLookAhead = True;

    # add these parameters as dict
    self.optimizer_input_dict = { "learning_rate" : learning_rate,
                                  "momentum" : momentum,                  # used by momentumGD
                                  "beta" : beta,                          # used by rmsprop
                                  "beta1" : beta1,                        # used by adam & nadam
                                  "beta2" : beta2,                        # used by adam & nadam
                                  "epsilon" : epsilon,
                                  "weight_decay" : weight_decay,
                                  "n_hiddenLayers": n_hiddenLayers}

    # weights and biases matrices
    self.weights = []
    self.biases = []
    self.lookAheadWeights = []
    self.lookAheadBiases = []

    self.wts_bias_history_dict = {"weights": self.weights, "biases": self.biases,
                                  "history_weights": [np.zeros(1) for _ in range(self.n_hiddenLayers+1)],         # these will be modified before their first use (dimensions of each values will also be changed)
                                  "history_biases": [np.zeros(1) for _ in range(self.n_hiddenLayers+1)],
                                  "second_history_weights": [np.zeros(1) for _ in range(self.n_hiddenLayers+1)],
                                  "second_history_biases": [np.zeros(1) for _ in range(self.n_hiddenLayers+1)],
                                  "dw": [np.empty(1) for _ in range(self.n_hiddenLayers+1)],
                                  "dh": [np.empty(1) for _ in range(self.n_hiddenLayers+1)]}

    self.initializeWeightsAndBiases()
    self.wts_bias_history_dict["second_history_weights"] = copy.deepcopy(self.wts_bias_history_dict["history_weights"])
    self.wts_bias_history_dict["second_history_biases"] = copy.deepcopy(self.wts_bias_history_dict["history_biases"])

    # pre-activation(a) and post-activation(h) values
    self.a = []
    self.h = []

  '''
    Weights,Biases initialization based on weight_init parameter

    weights[0]: input layer to first hidden layer  : input_size x n_neuronsPerLayer
    weights[1]: first hidden layer to second hidden layer : n_neuronsPerLayer x n_neuronsPerLayer
    ...
    weights[n_hiddenLayers]: last hidden layer to output layer : n_neuronsPerLayer x output_size

    biases[i] : bias for ith layer : 1 x n_neuronsPerLayer   (i:0 to n_hiddenLayers-1)
    biases[n_hiddenLayers]: 1 x output_size
  '''
  def initializeWeightsAndBiases(self):
    # biases for both types
    for i in range(self.n_hiddenLayers):
      self.biases.append(np.zeros(self.n_neuronsPerLayer))
      self.wts_bias_history_dict["history_biases"][i] = np.zeros(self.n_neuronsPerLayer)

    self.biases.append(np.zeros(self.output_size))   # biases[n_hiddenLayers]
    self.wts_bias_history_dict["history_biases"][self.n_hiddenLayers] = np.zeros(self.output_size)

    if(self.weight_init == "random"):   # Random Normal
      # weights[0]
      self.weights.append(np.random.randn(self.input_size, self.n_neuronsPerLayer))
      self.wts_bias_history_dict["history_weights"][0] = np.zeros((self.input_size, self.n_neuronsPerLayer))

      # weights[1] -> weights[n_hiddenLayers-1]
      for i in range(self.n_hiddenLayers-1):
        self.weights.append(np.random.randn(self.n_neuronsPerLayer, self.n_neuronsPerLayer))
        self.wts_bias_history_dict["history_weights"][i+1] = np.zeros((self.n_neuronsPerLayer, self.n_neuronsPerLayer))

      # weights[n_hiddenLayers]
      self.weights.append(np.random.randn(self.n_neuronsPerLayer, self.output_size))
      self.wts_bias_history_dict["history_weights"][self.n_hiddenLayers] = np.zeros((self.n_neuronsPerLayer, self.output_size))

    elif(self.weight_init == "Xavier"):   # Xavier Normal: mean = 0, variance = 2/(n_input + n_output)
      # weights[0]
      self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.input_size + self.n_neuronsPerLayer)), size=(self.input_size, self.n_neuronsPerLayer)))
      self.wts_bias_history_dict["history_weights"][0] = np.zeros((self.input_size, self.n_neuronsPerLayer))


      for i in range(self.n_hiddenLayers-1):
        self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.n_neuronsPerLayer + self.n_neuronsPerLayer)), size=(self.n_neuronsPerLayer, self.n_neuronsPerLayer)))
        self.wts_bias_history_dict["history_weights"][i+1] = np.zeros((self.n_neuronsPerLayer, self.n_neuronsPerLayer))


      self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.n_neuronsPerLayer + self.output_size)), size=(self.n_neuronsPerLayer, self.output_size)))
      self.wts_bias_history_dict["history_weights"][self.n_hiddenLayers] = np.zeros((self.n_neuronsPerLayer, self.output_size))

  '''
    Forward propagation through the neural network. (for batch)
    Instead of doing one input at a time, this function handles it for a batch using respective sized matrices

    x_batch: B x n where B - batch size, n- no of features = input_size
    x_batch is assumbed to be numpy array when given as input
  '''
  def forwardPropagation(self, x_batch, isValidation=False):
    a_pre_activation = []
    h_post_activation = []

    # considering a0,h0 as X values as a1: first layer  (it is calculated from x values)
    a_pre_activation.append(x_batch)
    h_post_activation.append(x_batch)

    wt = []
    b = []

    if(self.isLookAhead and not isValidation):
      for i in range(self.n_hiddenLayers+1):
        wt.append(self.weights[i] - (self.optimizer_input_dict["momentum"] * self.wts_bias_history_dict["history_weights"][i]))
        b.append(self.biases[i] - (self.optimizer_input_dict["momentum"] * self.wts_bias_history_dict["history_biases"][i]))

      self.lookAheadWeights = wt
      self.lookAheadBiases = b
    else:
      wt = copy.deepcopy(self.weights)
      b = copy.deepcopy(self.biases)

    # Except last layer since activation function could be different
    for i in range(self.n_hiddenLayers):
      # ai: B x n_neuronsPerLayer, biases[i]: 1 x n_neuronsPerLayer (it will be broadcasted while adding)
      ai = np.matmul(h_post_activation[-1], wt[i]) + b[i]
      hi = self.activationFun(ai)

      a_pre_activation.append(ai)
      h_post_activation.append(hi)

    # aL: last layer (activation function is softmax)
    aL = np.matmul(h_post_activation[-1], wt[self.n_hiddenLayers]) + b[self.n_hiddenLayers]
    hL = softmax(aL)   # y_batch

    a_pre_activation.append(aL)
    h_post_activation.append(hL)

    return a_pre_activation, h_post_activation

  '''
    Backward propagation through the neural network. (for batch)
  '''
  def backwardPropagation(self, a_pre_activation, h_post_activation, y_batch, y_pred_batch):
    grad_w = []
    grad_b = []
    grad_a = []
    grad_h = []

    # Output gradient (wrt aL)
    grad_hL = self.derivatesFuncMap[self.lossFunc.__name__](y_batch, y_pred_batch)
    grad_h.append(grad_hL)

    if(self.lossFunc.__name__ == "cross_entropy_loss"):
      grad_aL = y_pred_batch - y_batch    # just to reduce computation of jacobian matrix
      grad_a.append(grad_aL)
    else:
      grad_aL_list = []
      # softmax derivatives of each input is a matrix of size output_size x output_size, we need to perform matrix_mul for each input of batch
      for i in range(y_batch.shape[0]):   # self.batch_size = y_batch.shape[0] but better to take y_batch.shape[0] since last batch inputs can have less
        grad_aL_inp_i = grad_hL[i] @ softmax_derivative(y_pred_batch[i])
        grad_aL_list.append(grad_aL_inp_i)

      grad_aL = np.array(grad_aL_list)
      grad_a.append(grad_aL)                    # aL contains (aL) values of all inputs in the batch

    # Hidden layers
    for k in range(self.n_hiddenLayers, -1, -1):
      # gradients w.r.t parameters
      # wk
      grad_wk = np.zeros_like(self.weights[k])    # will be equal to sum across

      for inpNum in range(y_batch.shape[0]):
        grad_wk_inp_num = np.matmul(h_post_activation[k][inpNum].reshape(-1,1), grad_a[-1][inpNum].reshape(1,-1))
        grad_wk += grad_wk_inp_num
      grad_w.append(grad_wk)                   # contains sum across all batches

      # bk
      grad_bk = np.zeros_like(self.biases[k])
      for inpNum in range(y_batch.shape[0]):
        grad_bk += grad_a[-1][inpNum]
      grad_b.append(grad_bk)                     # contains sum across all batches

      if(k > 0):
        # gradients w.r.t layer below
        grad_hk_1 = grad_a[-1] @ self.weights[k].T
        grad_h.append(grad_hk_1)

        # gradients w.r.t layer below (pre-activation)
        grad_ak_1 = grad_hk_1 * self.derivatesFuncMap[self.activationFun.__name__](a_pre_activation[k])
        grad_a.append(grad_ak_1)

    grad_w = grad_w[::-1]
    grad_b = grad_b[::-1]

    return grad_w, grad_b

  def updateWeights(self, grad_w, grad_b, itr):
    self.wts_bias_history_dict["dw"] = grad_w
    self.wts_bias_history_dict["db"] = grad_b
    self.optimizer(self.optimizer_input_dict, self.wts_bias_history_dict, itr)


## Loading data

In [76]:
from keras.datasets import fashion_mnist, mnist
import numpy as np

datasets = {"fashion_mnist": fashion_mnist, "mnist": mnist}

def load_data(dataset_name):
  (x_train, y_train), (x_test, y_test) = datasets[dataset_name].load_data()
  num_classes = len(np.unique(y_train))

  y_train = np.eye(num_classes)[y_train]
  y_test = np.eye(num_classes)[y_test]

  x_train = x_train.reshape(x_train.shape[0], -1)
  x_test = x_test.reshape(x_test.shape[0], -1)

  x_train = x_train/255
  y_train = y_train/255

  return x_train, y_train, x_test, y_test, num_classes

## Sweep Configuration

In [77]:
sweep_configuration = {
    "method": "bayes",
    "metric": {"name": "validation_accuracy", "goal": "maximize"},
    "parameters": {
        "learning_rate": {"values": [1e-3, 1e-4]},
        "optimizer": {"values": ["sgd", "momentum", "nag", "rmsprop",  "adam", "nadam"]},
        "num_layers": {"values": [3, 4, 5]},
        "hidden_size": {"values": [32, 64, 128]},
        "batch_size": {"values": [16, 32, 64]},
        "activation": {"values": ["sigmoid", "tanh", "ReLU"]},
        "weight_decay": {"values": [0, 0.0005, 0.5]},
        "weight_init": {"values": ["random", "Xavier"]},
        "epochs": {"values": [5, 10]},
        "loss": {"values": ["cross_entropy"]},
        "momentum": {"values": [0.9]},
        "beta": {"values": [0.9]},
        "beta1": {"values":[0.9]},
        "beta2": {"values": [0.999]},
        "epsilon": {"values": [1e-8]},
        "dataset": {"values":["fashion_mnist"]}
    }
}

## Training the Feed Forward Neural Network

In [81]:
# train.py file (suppose parser gives all args to train network)
from sklearn.model_selection import train_test_split
import math

def calculateAccuracy(y_true, y_pred):
  y_pred_labels = np.argmax(y_pred, axis=1)
  y_true_labels = np.argmax(y_true, axis=1)
  accuracy = np.mean(y_pred_labels == y_true_labels)
  return accuracy

def trainNeuralNetwork():
  args = wandb.config
  x_train, y_train, x_test, y_test, num_classes = load_data(args["dataset"])
  input_size = len(x_train[0])
  output_size = num_classes
  n_hiddenLayers = args["num_layers"]
  n_neuronsPerLayer = args["hidden_size"]
  activationFun = args["activation"]
  weight_init = args["weight_init"]
  batch_size = args["batch_size"]
  lossFunc = args["loss"]
  optimizer = args["optimizer"]
  learning_rate = args["learning_rate"]
  momentum = args["momentum"]
  beta = args["beta"]
  beta1 = args["beta1"]
  beta2 = args["beta2"]
  epsilon = args["epsilon"]
  weight_decay = args["weight_decay"]
  epochs = args["epochs"]

  wandb.run.name = f"{optimizer}_{activationFun}_{n_hiddenLayers}_{n_neuronsPerLayer}_{epochs}_{weight_init}_{lossFunc}"

  # paste all above paramters as fun params
  fnn = FeedForwardNeuralNetwork(input_size, output_size, n_hiddenLayers, n_neuronsPerLayer,
                                 activationFun, weight_init, batch_size, lossFunc,
                                 optimizer, learning_rate, momentum,
                                 beta, beta1, beta2,
                                 epsilon, weight_decay, epochs)

  x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.1, random_state=42)
  num_batches = math.ceil(len(x_train)/batch_size)

  for epochNum in range(epochs):
    for batchNum in range(num_batches):
      start_idx = batchNum * batch_size
      end_idx = start_idx + batch_size

      x_batch = x_train[start_idx:end_idx]
      y_batch = y_train[start_idx:end_idx]

      # Forward Propagation
      a_pre_activation, h_post_activation = fnn.forwardPropagation(x_batch)
      y_pred_batch = h_post_activation[-1]

      # Back Propagation
      grad_w, grad_b = fnn.backwardPropagation(a_pre_activation, h_post_activation, y_batch, y_pred_batch)

      # Update weights
      itr = epochNum * num_batches + batchNum + 1
      fnn.updateWeights(grad_w, grad_b, itr)

    # Validation accuracy
    _, h_validation = fnn.forwardPropagation(x_validation, isValidation=True)
    y_pred_validation = h_validation[-1]
    validation_accuracy = calculateAccuracy(y_validation, y_pred_validation)

    # Train accuracy
    _, h_train = fnn.forwardPropagation(x_train, isValidation=True)
    y_pred_train = h_train[-1]
    train_accuracy = calculateAccuracy(y_train, y_pred_train)

    wandb.log({
        "epoch": epochNum + 1,
        "validation_loss": fnn.lossFunc(y_batch, y_pred_batch),
        "validation_accuracy": validation_accuracy,
        "train_loss": fnn.lossFun(y_batch, y_pred_batch),
        "train_accuracy": train_accuracy
        })

  # Test accuracy
  _,h_test = fnn.forwardPropagation(x_test, isValidation=True)
  y_pred_test = h_test[-1]
  test_accuracy = calculateAccuracy(y_test, y_pred_test)
  wandb.log(f"Run Summary for {wandb.run.name}")
  wandb.log({"test_accuracy": test_accuracy,
             "test_loss": fnn.lossFunc(y_test, y_pred_test)})

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7feb04572cd0>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7feb04572cd0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [79]:
wandb_id = wandb.sweep(sweep_configuration, project="DA6401_Assignment1")
wandb.agent(wandb_id, function=trainNeuralNetwork, count=2)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: xzwni6rg
Sweep URL: https://wandb.ai/nikhithaa-iit-madras/DA6401_Assignment1/sweeps/xzwni6rg


wandb: Agent Starting Run: nrkz5e0h with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dataset: fashion_mnist
wandb: 	epochs: 10
wandb: 	epsilon: 1e-08
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	loss: cross_entropy
wandb: 	momentum: 0.9
wandb: 	num_layers: 4
wandb: 	optimizer: nag
wandb: 	weight_decay: 0
wandb: 	weight_init: Xavier
Exception in thread Thread-65 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-78-aad1e9687f1b>", line 12, in trainNeuralNetwork
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1485, in init
    wandb._sentry.reraise(e)
  File "/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py", line 156, in reraise
    raise exc.with_traceback(sys.exc_info()[2])
  File "/usr/local/lib/python3.11/dist

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7feb04572cd0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7feb04572cd0>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7feb04572cd0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

## Random tests

In [6]:
import numpy as np

# # Create a column vector (shape: (3,1))
# v1 = np.array([[1,2], [2,3], [3,4]])  # Shape (3,1)
# v2 = np.array([[4], [5], [6]])  # Shape (3,1)

# # Append to a list
# vector_list = []
# vector_list.append(v1)
# vector_list.append(v2)

# print(vector_list)

# # flatten each element of x_train, after flattening store it as np array
# x_ftrain = np.array([x.flatten() for x in x_train])
# x1 = [x.flatten() for x in x_train]
# # print(x_ftrain[0].shape)
# # print(x_ftrain.shape)
# # print(x_ftrain[:20].shape)
# # print(y_train[:20].shape)
# # print(x_test.shape)
# # print(y_test.shape)
# m = np.array(np.zeros((784,3)))
# y = np.matmul(x_ftrain[:20],m)
# print(y.shape)


y_batch = np.array([
    [0, 1, 0.5],  # Example 1
    [1, 0, 0],  # Example 2
    [0, 0, 1]   # Example 3
])

y_hat_batch = np.array([
    [0.2, 0.5, 0.3],
    [0.8, 0.1, 0.1],
    [0.3, 0.3, 0.4]
])

grad_batch = - (y_batch / y_hat_batch)  # Element-wise division
print(grad_batch)


[[-0.         -2.         -1.66666667]
 [-1.25       -0.         -0.        ]
 [-0.         -0.         -2.5       ]]


In [ ]:
import numpy as np

def identity(x):
    return x

def identity_derivative(x):
    return np.ones_like(x)

x_single = 2.0
x_vector = np.array([1.0, 2.0, -3.0])
x_batch = np.array([[1.0, 1.0], [3.0, 4.0]])

print(identity(x_batch))  # Works
print(identity_derivative(x_batch))  # Works


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

print(sigmoid(x_vector))  # Works
print(sigmoid_derivative(x_vector))  # Works

In [ ]:
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

print(tanh(x_vector))  # Works
print(tanh_derivative(x_single))  # Works

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

print(relu(x_vector))  # Works
print(relu_derivative(x_vector))  # Works

In [74]:
# # def softmax(x):
# #     exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Stability trick
# #     return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# # Softmax needs a vector input
# x_vector = np.array([1.0, 20000.0, 3.0])
# x_batch = np.array([[1.0, 2.0, 3000.0], [1.5, 2.5, 3.5]])

# print(softmax(x_vector))  # Works
# print(softmax(x_batch))  # Works (batch-wise softmax)

In [39]:
print(y_train)

[9 0 0 ... 3 0 5]


In [75]:
import numpy as np

num_classes = 10
y_train_one_hot = np.eye(num_classes)[y_train]
y_test_one_hot = np.eye(num_classes)[y_test]

print(y_train_one_hot[-1])  # Output: (num_samples, 10)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [95]:
print(x_vector)
x_vector=[1,2,3]

[ 1.  2. -3.]


In [110]:
y_true_single = np.array([0, 0, 1])  # One-hot encoded true label
y_pred_single = np.array([0.1, 0.7, 0.2])
cross_entropy_loss_derivative(y_true_single, y_pred_single)

y_true_batch = np.array([[0, 1, 0], [1, 0, 0]])  # One-hot encoded true labels for two examples
y_pred_batch = np.array([[0.1, 0.7, 0.2], [0.8, 0.1, 0.1]])
cross_entropy_loss_derivative(y_true_batch, y_pred_batch)

array([[ 0.        , -1.42857143,  0.        ],
       [-1.25      ,  0.        ,  0.        ]])

In [ ]:
x = softmax_derivative(y_pred_single)
print(x)
print(x.shape)

y = softmax_derivative(y_pred_batch)
print(y)
print(y[0].shape)
print(y.shape)
# len(y_true_batch.shape)

In [136]:
def f(dictx):
  dictx["key"] = 1
  dictx["dgy"] = 2
  print(dictx)

dicty = {}
f(dicty)
print(dicty)
dicty["2"] = 2
f(dicty)

{'key': 1, 'dgy': 2}
{'key': 1, 'dgy': 2}
{'key': 1, 'dgy': 2, '2': 2}


In [3]:
import numpy as np

def cross_entropy_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred), axis=-1)

def cross_entropy_loss_derivative(y_true, y_pred):
    return -y_true / y_pred  # Element-wise division

def softmax_derivative(inp: np.array):
    derivates = []
    if len(inp.shape) == 1:
        S_vector = inp.reshape(-1, 1)
        derivates = np.diag(inp) - np.dot(S_vector, S_vector.T)
    elif len(inp.shape) == 2:
        for i in range(inp.shape[0]):
            S_vector = inp[i].reshape(-1, 1)
            derivates.append(np.diag(inp[i]) - np.dot(S_vector, S_vector.T))
    return np.array(derivates)

# Sample data
y_true = np.array([[0, 1, 0], [1, 0, 0]])  # One-hot encoded true labels for 2 samples, 3 classes
y_pred = np.array([[0.1, 0.8, 0.1], [0.7, 0.2, 0.1]])  # Predicted probabilities from softmax

# Derivatives
grad_hL = cross_entropy_loss_derivative(y_true, y_pred)  # Gradient of cross-entropy loss
print(grad_hL)

# Assume the activation function is softmax, and we have computed the pre-activation (a_pre_activation[-1])
# Use softmax derivative for the last layer
for i in range(y_true.shape[0]):
    grad_aL = grad_hL[i] @ softmax_derivative(y_pred[i])  # Element-wise product
    print(grad_aL)
# grad_aL = grad_hL * softmax_derivative(y_pred)  # Element-wise product
# print(grad_aL)


[[ 0.         -1.25        0.        ]
 [-1.42857143  0.          0.        ]]
[ 0.1 -0.2  0.1]
[-0.3  0.2  0.1]


#  x

In [28]:
print(type(x_train))
x_train = x_train.reshape(x_train.shape[0], -1)
print(len(x_train[0]))

<class 'numpy.ndarray'>
784


In [30]:
x =[1,2,3,4]
# x[:7]

4